In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.metrics import r2_score
import math
import pandas as pd

In [3]:
#Delete NaN values
def del_nans(data):
    nans = data.isnull().values.T
    nans_column = []

    for i in range(len(nans)):
        if True in nans[i]:
            nans_column.append(i)

    for i in nans_column:
        for j in range(len(data)):
            if math.isnan(data.iloc[j, i]):
                data.iloc[j, i] = 0

In [4]:
def set_data():
    #Params and replies for train
    path_train_data = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
    path_test_data = '/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
    path_submission_data = '/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv'
    
    train_data = pd.read_csv(path_train_data)
    test_data = pd.read_csv(path_test_data)
    submission_data = pd.read_csv(path_submission_data)
    
    train_param = train_data.iloc[:,1:-1]
    train_reply = train_data.iloc[:,-1]
    test_param = test_data.iloc[:,1:]
    test_reply = submission_data.iloc[:,-1]
    
    return test_param, test_reply, train_param, train_reply

In [5]:
def normalization_x(data):
    for i in range(data.shape[1]):
        data[:, i] = (data[:, i] - np.mean(data[:, i])) / np.var(data[:, i])

In [6]:
def normalization_y(data):
    data = data / np.max(data)
    
    return data

In [7]:
def preprocessing_y(data):
    preprocessing_data = np.array(data, dtype=np.float64)
    preprocessing_data = normalization_y(preprocessing_data)
    
    return preprocessing_data

In [8]:
def preprocessing_x(data):
    preprocessing_data = data
    
    del_nans(preprocessing_data)
    preprocessing_data = np.array(data, dtype=np.float64)
    normalization_x(preprocessing_data)
    
    return preprocessing_data

In [9]:
def electrical_handler(data):
    new_data = data.replace({'SBrkr': 0, 'FuseA': 1, 'FuseF': 2, 'FuseP': 3, 'Mix': 4})
    
    return new_data

In [10]:
test_param, test_reply, train_param, train_reply = set_data()

features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Electrical']

train_X = train_param[features]
train_X = electrical_handler(train_X)
train_X  = preprocessing_x(train_X)

train_y = preprocessing_y(train_reply)

test_X = test_param[features]
test_X = electrical_handler(test_X)
test_X  = preprocessing_x(test_X)

test_y = preprocessing_y(test_reply)

In [11]:
regressor = SVR(kernel='linear')
regressor.fit(train_X, train_y)

SVR(kernel='linear')

In [12]:
val_predictions = regressor.predict(test_X)
val_mae = r2_score(val_predictions, test_y)
print(val_mae)

-23.054694746918823
